## **1. Basic Model**

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import random


# 파라미터 설정 (learning rate, training epochs, batch_size)
learning_rate = 0.1
training_epochs = 15
batch_size = 100


#train과 test set으로 나누어 MNIST data 불러오기

mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)


#dataset loader에 train과 test할당하기(batch size, shuffle, drop_last 잘 설정할 것!)

train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True)


# Layer 쌓기 (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기)

# 각 Layer의 Hidden node 수 : 1st Layer (784,100), 2nd Layer(100,100),3rd Layer(100,10)

linear1 = torch.nn.Linear(784, 100, bias=True)
linear2 = torch.nn.Linear(100, 100, bias=True)
linear3 = torch.nn.Linear(100, 10, bias=True)
relu = torch.nn.ReLU()
bn1 = torch.nn.BatchNorm1d(100)
bn2 = torch.nn.BatchNorm1d(100)
dropout = torch.nn.Dropout(p=0.3)


#xavier initialization을 이용하여 각 layer의 weight 초기화

torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)


#model 정의하기(쌓는 순서: linear-Batch Normalization layer - ReLU- DropOut)
bn_model = torch.nn.Sequential(linear1, bn1, relu, dropout,
                            linear2, bn2, relu,dropout,
                            linear3)

# Loss Function 정의하기 (CrossEntropy를 사용할 것!)
criterion = torch.nn.CrossEntropyLoss()

#optimizer 정의하기 (Adam optimizer를 사용할 것!)
bn_optimizer = torch.optim.Adam(bn_model.parameters(), lr=learning_rate)



#cost 계산을 위한 변수 설정
train_total_batch = len(train_loader)


#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것) 
for epoch in range(training_epochs):
    avg_cost = 0
    bn_model.train()  # set the model to train mode

    
    #train dataset을 불러오고, back propagation을 통해 loss를 최적화하는 과정
    for X, Y in train_loader:
        X = X.view(-1, 28 * 28)
        Y = Y

        bn_optimizer.zero_grad()
        bn_prediction = bn_model(X)
        bn_loss = criterion(bn_prediction, Y)
        bn_loss.backward()
        bn_optimizer.step()

        avg_cost += bn_loss / train_total_batch
	 
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')



#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 .test_labels 사용
#accuracy의 초기 값 설정(0으로) 꼭 할 것

with torch.no_grad():
    bn_model.eval()     
    X_test = mnist_test.test_data.view(-1, 28 * 28).float()
    Y_test = mnist_test.test_labels
    
    bn_acc = 0 
    bn_prediction = bn_model(X_test)
    bn_correct_prediction = torch.argmax(bn_prediction, 1) == Y_test
    bn_loss += criterion(bn_prediction, Y_test)
    bn_acc += bn_correct_prediction.float().mean()
    
    print("Accuracy: ", bn_acc.item())
    
    
    
    ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = bn_model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Epoch: 0001 cost = 0.507042050
Epoch: 0002 cost = 0.363651603
Epoch: 0003 cost = 0.347278297
Epoch: 0004 cost = 0.317302138
Epoch: 0005 cost = 0.303611279
Epoch: 0006 cost = 0.289466292
Epoch: 0007 cost = 0.276389003
Epoch: 0008 cost = 0.276228815
Epoch: 0009 cost = 0.257342339
Epoch: 0010 cost = 0.269999474
Epoch: 0011 cost = 0.265435487
Epoch: 0012 cost = 0.250227362
Epoch: 0013 cost = 0.250091642
Epoch: 0014 cost = 0.249411538
Epoch: 0015 cost = 0.227579162
Learning finished
Accuracy:  0.9211999773979187
Label:  4
Prediction:  4


C:\Users\DongHwi Kim\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\DongHwi Kim\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


## **2. Increasing Model**

In [2]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import random


# 파라미터 설정 (learning rate, training epochs, batch_size)
learning_rate = 0.1
training_epochs = 15
batch_size = 100


#train과 test set으로 나누어 MNIST data 불러오기

mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)


#dataset loader에 train과 test할당하기(batch size, shuffle, drop_last 잘 설정할 것!)

train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True)


# Layer 쌓기 (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기)

# 각 Layer의 Hidden node 수 : 1st Layer (784,100), 2nd Layer(100,100),3rd Layer(100,10)

linear1 = torch.nn.Linear(784, 200, bias=True)
linear2 = torch.nn.Linear(200, 200, bias=True)
linear3 = torch.nn.Linear(200, 10, bias=True)
relu = torch.nn.ReLU()
bn1 = torch.nn.BatchNorm1d(200)
bn2 = torch.nn.BatchNorm1d(200)
dropout = torch.nn.Dropout(p=0.3)


#xavier initialization을 이용하여 각 layer의 weight 초기화

torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)


#model 정의하기(쌓는 순서: linear-Batch Normalization layer - ReLU- DropOut)
bn_model = torch.nn.Sequential(linear1, bn1, relu, dropout,
                            linear2, bn2, relu,dropout,
                            linear3)

# Loss Function 정의하기 (CrossEntropy를 사용할 것!)
criterion = torch.nn.CrossEntropyLoss()

#optimizer 정의하기 (Adam optimizer를 사용할 것!)
bn_optimizer = torch.optim.Adam(bn_model.parameters(), lr=learning_rate)



#cost 계산을 위한 변수 설정
train_total_batch = len(train_loader)


#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것) 
for epoch in range(training_epochs):
    avg_cost = 0
    bn_model.train()  # set the model to train mode

    
    #train dataset을 불러오고, back propagation을 통해 loss를 최적화하는 과정
    for X, Y in train_loader:
        X = X.view(-1, 28 * 28)
        Y = Y

        bn_optimizer.zero_grad()
        bn_prediction = bn_model(X)
        bn_loss = criterion(bn_prediction, Y)
        bn_loss.backward()
        bn_optimizer.step()

        avg_cost += bn_loss / train_total_batch
	 
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')



#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 .test_labels 사용
#accuracy의 초기 값 설정(0으로) 꼭 할 것

with torch.no_grad():
    bn_model.eval()     
    X_test = mnist_test.test_data.view(-1, 28 * 28).float()
    Y_test = mnist_test.test_labels
    
    bn_acc = 0 
    bn_prediction = bn_model(X_test)
    bn_correct_prediction = torch.argmax(bn_prediction, 1) == Y_test
    bn_loss += criterion(bn_prediction, Y_test)
    bn_acc += bn_correct_prediction.float().mean()
    
    print("Accuracy: ", bn_acc.item())
    
    
    
    ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = bn_model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())
    



Epoch: 0001 cost = 0.466080725
Epoch: 0002 cost = 0.327673644
Epoch: 0003 cost = 0.292566210
Epoch: 0004 cost = 0.271716744
Epoch: 0005 cost = 0.261581928
Epoch: 0006 cost = 0.241671979
Epoch: 0007 cost = 0.240976557
Epoch: 0008 cost = 0.231926531
Epoch: 0009 cost = 0.229610890
Epoch: 0010 cost = 0.217063710
Epoch: 0011 cost = 0.211958751
Epoch: 0012 cost = 0.214407966
Epoch: 0013 cost = 0.204376295
Epoch: 0014 cost = 0.201951459
Epoch: 0015 cost = 0.198006734
Learning finished
Accuracy:  0.8970000147819519
Label:  7
Prediction:  7


## **3. Decreasing Model**

In [4]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pylab as plt
import random


# 파라미터 설정 (learning rate, training epochs, batch_size)
learning_rate = 0.1
training_epochs = 15
batch_size = 100


#train과 test set으로 나누어 MNIST data 불러오기

mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)


#dataset loader에 train과 test할당하기(batch size, shuffle, drop_last 잘 설정할 것!)

train_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=mnist_test,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True)


# Layer 쌓기 (조건: 3개의 Layer 사용, DropOut 사용 (p=0.3), ReLU 함수 사용, Batch normalization하기)

# 각 Layer의 Hidden node 수 : 1st Layer (784,100), 2nd Layer(100,100),3rd Layer(100,10)

linear1 = torch.nn.Linear(784, 50, bias=True)
linear2 = torch.nn.Linear(50, 50, bias=True)
linear3 = torch.nn.Linear(50, 10, bias=True)
relu = torch.nn.ReLU()
bn1 = torch.nn.BatchNorm1d(50)
bn2 = torch.nn.BatchNorm1d(50)
dropout = torch.nn.Dropout(p=0.3)


#xavier initialization을 이용하여 각 layer의 weight 초기화

torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)


#model 정의하기(쌓는 순서: linear-Batch Normalization layer - ReLU- DropOut)
bn_model = torch.nn.Sequential(linear1, bn1, relu, dropout,
                            linear2, bn2, relu,dropout,
                            linear3)

# Loss Function 정의하기 (CrossEntropy를 사용할 것!)
criterion = torch.nn.CrossEntropyLoss()

#optimizer 정의하기 (Adam optimizer를 사용할 것!)
bn_optimizer = torch.optim.Adam(bn_model.parameters(), lr=learning_rate)



#cost 계산을 위한 변수 설정
train_total_batch = len(train_loader)


#Training epoch (cost 값 초기 설정(0으로)과 model의 train 설정 꼭 할 것) 
for epoch in range(training_epochs):
    avg_cost = 0
    bn_model.train()  # set the model to train mode

    
    #train dataset을 불러오고, back propagation을 통해 loss를 최적화하는 과정
    for X, Y in train_loader:
        X = X.view(-1, 28 * 28)
        Y = Y

        bn_optimizer.zero_grad()
        bn_prediction = bn_model(X)
        bn_loss = criterion(bn_prediction, Y)
        bn_loss.backward()
        bn_optimizer.step()

        avg_cost += bn_loss / train_total_batch
	 
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')



#test data로 모델의 정확도를 검증하는 코드 (model의 evaluation mode 설정 꼭 할 것)
#X_test 불러올 때 view를 사용하여 차원 변환할 것/ Y_test를 불러올때 .test_labels 사용
#accuracy의 초기 값 설정(0으로) 꼭 할 것

with torch.no_grad():
    bn_model.eval()     
    X_test = mnist_test.test_data.view(-1, 28 * 28).float()
    Y_test = mnist_test.test_labels
    
    bn_acc = 0 
    bn_prediction = bn_model(X_test)
    bn_correct_prediction = torch.argmax(bn_prediction, 1) == Y_test
    bn_loss += criterion(bn_prediction, Y_test)
    bn_acc += bn_correct_prediction.float().mean()
    
    print("Accuracy: ", bn_acc.item())
    
    
    
    ##Test set에서 random으로 data를 뽑아 Label과 Prediction을 비교하는 코드 
    r = random.randint(0, len(mnist_test)-1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 *28).float()
    Y_single_data = mnist_test.test_labels[r:r + 1]
    
    print('Label: ', Y_single_data.item())
    single_prediction = bn_model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Epoch: 0001 cost = 0.593761981
Epoch: 0002 cost = 0.447104007
Epoch: 0003 cost = 0.401066989
Epoch: 0004 cost = 0.390282810
Epoch: 0005 cost = 0.376429737
Epoch: 0006 cost = 0.386880755
Epoch: 0007 cost = 0.370123804
Epoch: 0008 cost = 0.359081507
Epoch: 0009 cost = 0.353553027
Epoch: 0010 cost = 0.351909220
Epoch: 0011 cost = 0.334892690
Epoch: 0012 cost = 0.343113780
Epoch: 0013 cost = 0.337895274
Epoch: 0014 cost = 0.334494233
Epoch: 0015 cost = 0.335095257
Learning finished
Accuracy:  0.9415000081062317
Label:  7
Prediction:  0


### **결과 분석**

Hidden node의 개수를 늘리면 train에서의 cost는 줄어들고, test set에서의 Accuracy도 줄어든다.

Hidden node의 개수를 줄이면 train에서의 cost는 증가하고, test set에서의 Accuracy도 증가한다.